# NLP y NLTK Basicos
---

## SparkContext y SparkSession

In [1]:
from pyspark import SparkContext
sc = SparkContext(master = 'local')

from pyspark.sql import SparkSession
spark = SparkSession.builder \
          .appName("Python Spark SQL basic example") \
          .config("spark.some.config.option", "some-value") \
          .getOrCreate()

Muchos de los ejemplos de este artículo han sido tomados del libro escrito por **Bird et al. (2009)**. Aquí traté de poner en práctica los ejemplos del libro con la mayor chispa posible.

Consulte el libro para más detalles: Bird, Steven, Ewan Klein y Edward Loper. Procesamiento de lenguaje natural con Python: análisis de texto con el kit de herramientas de lenguaje natural. "O'Reilly Media, Inc.", 2009.

## Terminología básica

* **text**: una secuencia de palabras y puntuación.
* **frequency distribution**: la frecuencia de las palabras en un objeto de texto.
* **collocation**: a **sequence of words** que ocurren juntos inusualmente a menudo.
* **bigrams**: pares de palabras. Los bigrams de alta frecuencia son colocaciones.
* **corpus**: un gran cuerpo de texto
* **wordnet**: una base de datos léxica en la que las palabras en inglés se agrupan en conjuntos de sinónimos (**también llamados synsets**).
* **text normalization**: el proceso de transformar el texto en una sola forma canónica, por ejemplo, convertir el texto en minúsculas, eliminar las puntuaciones, etc.
* **Lemmatization**: el proceso de agrupar formas de variantes de la misma palabra para que puedan analizarse como una única posición.
* **Stemming**: el proceso de reducir las palabras inflexionadas a su **tronco de palabras**.
* **tokenization**:
* **segmentation**:
* **chunking**:

## Textos como listas de palabras

Cree un marco de datos compuesto de elementos de texto.

In [3]:
import pandas as pd
pdf = pd.DataFrame({
        'texts': [['I', 'like', 'playing', 'basketball'],
                 ['I', 'like', 'coding'],
                 ['I', 'like', 'machine', 'learning', 'very', 'much']]
    })
    
df = spark.createDataFrame(pdf)
df.show(truncate=False)

+----------------------------------------+
|texts                                   |
+----------------------------------------+
|[I, like, playing, basketball]          |
|[I, like, coding]                       |
|[I, like, machine, learning, very, much]|
+----------------------------------------+



## Ngramas y coubicaciones

Transformar textos en colocaciones de 2, 3 y 4 gramos.

In [4]:
from pyspark.ml.feature import NGram
from pyspark.ml import Pipeline
ngrams = [NGram(n=n, inputCol='texts', outputCol=str(n)+'-grams') for n in [2,3,4]]

# construir modelo pipeline 
pipeline = Pipeline(stages=ngrams)

# transformar datos
texts_ngrams = pipeline.fit(df).transform(df)

In [5]:
# mostrar resultado
texts_ngrams.select('2-grams').show(truncate=False)
texts_ngrams.select('3-grams').show(truncate=False)
texts_ngrams.select('4-grams').show(truncate=False)

+------------------------------------------------------------------+
|2-grams                                                           |
+------------------------------------------------------------------+
|[I like, like playing, playing basketball]                        |
|[I like, like coding]                                             |
|[I like, like machine, machine learning, learning very, very much]|
+------------------------------------------------------------------+

+----------------------------------------------------------------------------------+
|3-grams                                                                           |
+----------------------------------------------------------------------------------+
|[I like playing, like playing basketball]                                         |
|[I like coding]                                                                   |
|[I like machine, like machine learning, machine learning very, learning very much]|
+-----

## Acceso a los corpus desde el paquete NLTK

### El corpus `gutenberg`

#### Obtener identificadores de archivos en gutenberg corpus

In [6]:
from nltk.corpus import gutenberg

gutenberg_fileids = gutenberg.fileids()
gutenberg_fileids

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

#### Vía de acceso absoluta de un fichero

In [7]:
gutenberg.abspath(gutenberg_fileids[0])

FileSystemPathPointer('/Users/mingchen/nltk_data/corpora/gutenberg/austen-emma.txt')

#### Raw text

In [8]:
gutenberg.raw(gutenberg_fileids[0])[:200]

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; an'

#### Las palabras de todo el corpus

In [9]:
gutenberg.words()

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', ...]

In [10]:
len(gutenberg.words())

2621613

#### Sentencias de un expediente específico

In [11]:
gutenberg.sents(gutenberg_fileids[0])

[['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']'], ['VOLUME', 'I'], ...]

In [12]:
len(gutenberg.sents(gutenberg_fileids[0]))

7752

### Carga de corpus personalizado

Vamos a crear un corpus que contenga todos los archivos del directorio **./data**.

In [13]:
from nltk.corpus import PlaintextCorpusReader
corpus_data = PlaintextCorpusReader('./data', '.*')

#### Archivos en el corpus *corpus_data*

In [14]:
data_fileids = corpus_data.fileids()
data_fileids

['Advertising.csv',
 'Credit.csv',
 'WineData.csv',
 'churn-bigml-20.csv',
 'churn-bigml-80.csv',
 'cuse_binary.csv',
 'horseshoe_crab.csv',
 'hsb2.csv',
 'hsb2_modified.csv',
 'iris.csv',
 'mtcars.csv',
 'prostate.csv',
 'twitter.txt']

#### Raw text en *twitter.txt*

In [15]:
corpus_data.raw('twitter.txt')

'Fresh install of XP on new computer. Sweet relief! fuck vista\t1018769417\t1.0\nWell. Now I know where to go when I want my knives. #ChiChevySXSW http://post.ly/RvDl\t10284216536\t1.0\n"Literally six weeks before I can take off ""SSC Chair"" off my email. Its like the torturous 4th mile before everything stops hurting."\t10298589026\t1.0\nMitsubishi i MiEV - Wikipedia, the free encyclopedia - http://goo.gl/xipe Cutest car ever!\t109017669432377344\t1.0\n\'Cheap Eats in SLP\' - http://t.co/4w8gRp7\t109642968603963392\t1.0\nTeenage Mutant Ninja Turtle art is never a bad thing... http://bit.ly/aDMHyW\t10995492579\t1.0\nNew demographic survey of online video viewers: http://bit.ly/cx8b7I via @KellyOlexa\t11713360136\t1.0\nhi all - i\'m going to be tweeting things lookstat at the @lookstat twitter account. please follow me there\t1208319583\t1.0\nHoly carp, no. That movie will seriously suffer for it. RT @MouseInfo: Anyone excited for The Little Mermaid in 3D?\t121330835726155776\t1.0\n"Di

#### Palabras y frases en archivo *twitter.txt*

In [16]:
corpus_data.words(fileids='twitter.txt')

['Fresh', 'install', 'of', 'XP', 'on', 'new', ...]

In [17]:
len(corpus_data.words(fileids='twitter.txt'))

253

In [18]:
corpus_data.sents(fileids='twitter.txt')

[['Fresh', 'install', 'of', 'XP', 'on', 'new', 'computer', '.'], ['Sweet', 'relief', '!'], ...]

In [19]:
len(corpus_data.sents(fileids='twitter.txt'))

14

## WordNet

La función `nltk.corpus.wordnet.synsets()` carga todos los sintetizadores con un determinado lema y parte de la etiqueta de voz.

Carga todos los synsets en un marco de datos de chispa dado el lema `car`.

In [20]:
from nltk.corpus import wordnet
wordnet.synsets
pdf = pd.DataFrame({
        'car_synsets': [synsets._name for synsets in wordnet.synsets('car')]
    })
df = spark.createDataFrame(pdf)
df.show()

+--------------+
|   car_synsets|
+--------------+
|      car.n.01|
|      car.n.02|
|      car.n.03|
|      car.n.04|
|cable_car.n.01|
+--------------+



### Obtener nombres de lemas a los que se les ha dado un synset

In [41]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from nltk.corpus import wordnet

def lemma_names_from_synset(x):
    synset = wordnet.synset(x)
    return synset.lemma_names()

lemma_names_from_synset('car.n.02')
# synset_lemmas_udf = udf(lemma_names_from_synset, ArrayType(StringType()))


['car', 'railcar', 'railway_car', 'railroad_car']